## Pizza Classifier
Determines whether an image contains a pizza<br>
Created for CS50<br>
Dataset: https://www.kaggle.com/datasets/carlosrunner/pizza-not-pizza

In [1]:
import cv2
import cupy as np
from importlib import reload
from os import listdir
from sandbox import model, layers, activations, costs, utils

In [2]:
# Load altered pizza dataset
dataset_path = 'dataset\\altered\\'
datasets = [
    np.array([
        cv2.imread(dataset_path + dir + img).flatten()
        for img in listdir(dataset_path + dir)
    ])
    for dir in ['train\\pizza\\', 'train\\not_pizza\\', 'test\\pizza\\', 'test\\not_pizza\\']
]

# Create training and testing data
train_x = np.concatenate((datasets[0], datasets[1]), axis=0) / 255
train_y = np.concatenate((np.ones((datasets[0].shape[0], 1)), np.zeros((datasets[1].shape[0], 1))), axis=0)

test_x = np.concatenate((datasets[2], datasets[3]), axis=0) / 255
test_y = np.concatenate((np.ones((datasets[2].shape[0], 1)), np.zeros((datasets[3].shape[0], 1))), axis=0)

In [3]:
reload(model)

# Create model
pizza = model.Model(cuda=True)
pizza.add(layers.Dense(units=32, activation=activations.ReLU()))
pizza.add(layers.Dense(units=16, activation=activations.ReLU()))
pizza.add(layers.Dense(units=8, activation=activations.ReLU()))
pizza.add(layers.Dense(units=1, activation=activations.Sigmoid()))
pizza.configure(learning_rate=0.01, epochs=2500, cost_type=costs.BinaryCrossentropy())

# Train model
pizza.train(train_x, train_y, verbose=True)

Cost on epoch 0: 0.91484
Cost on epoch 100: 0.62681
Cost on epoch 200: 0.61044
Cost on epoch 300: 0.60031
Cost on epoch 400: 0.59088
Cost on epoch 500: 0.58531
Cost on epoch 600: 0.57721
Cost on epoch 700: 0.56762
Cost on epoch 800: 0.5659
Cost on epoch 900: 0.55642
Cost on epoch 1000: 0.55207
Cost on epoch 1100: 0.54619
Cost on epoch 1200: 0.54417
Cost on epoch 1300: 0.53825
Cost on epoch 1400: 0.53711
Cost on epoch 1500: 0.52731
Cost on epoch 1600: 0.51697
Cost on epoch 1700: 0.51649
Cost on epoch 1800: 0.51188
Cost on epoch 1900: 0.50519
Cost on epoch 2000: 0.50181
Cost on epoch 2100: 0.49348
Cost on epoch 2200: 0.48793
Cost on epoch 2300: 0.48144
Cost on epoch 2400: 0.47576
Cost on epoch 2500: 0.4695


In [6]:
# Assess model accuracy
pred_train = pizza.predict(train_x, prediction_type=utils.Predictions.binary_classification) # Get model accuracy on training data
print('Training Accuracy: ' + str(np.round(np.sum((pred_train == train_y)/train_x.shape[0]), decimals=5)))
pred_test = pizza.predict(test_x, prediction_type=utils.Predictions.binary_classification) # Get model accuracy on testing data
print('Testing Accuracy: ' + str(np.round(np.sum((pred_test == test_y)/test_x.shape[0]), decimals=5)))

Training Accuracy: 0.78907
Testing Accuracy: 0.70833
